In [1]:
import sys
sys.path.append('structural-probes')
sys.path.append('finetuning')
from pathlib import Path

from run_experiment import setup_new_experiment_dir, execute_experiment
import yaml
import torch
import pandas as pd
import eval_probes_on_dataset
import jupyter_slack

from transformers import BertConfig, BertTokenizer, BertForSequenceClassification

/home/mli/anaconda3/envs/data-mining/lib/python3.6/site-packages/jupyter_slack/jupyter_slack.py:18: UserWarning: Either $SLACK_WEBHOOK_URL must be set (see https://api.slack.com/messaging/webhooks) or both $SLACK_TOKEN and $SLACK_ID must be set (see https://api.slack.com/custom-integrations/legacy-tokens). All notifications will be noops under the current setting.
  "Either $SLACK_WEBHOOK_URL must be set "


In [2]:
def setup_args_and_folder(): 
    CONFIG_FILE = 'example/config/bert_ptb3.yaml'
    EXPERIMENT_NAME = ''
    SEED = 123

    class Object(object):
        pass

    cli_args = Object()
    cli_args.experiment_config = CONFIG_FILE
    cli_args.results_dir = EXPERIMENT_NAME
    cli_args.train_probe = -1
    cli_args.report_results = 1
    cli_args.seed = SEED

    yaml_args = yaml.load(open(cli_args.experiment_config))
    setup_new_experiment_dir(cli_args, yaml_args, cli_args.results_dir)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    yaml_args['device'] = device
    yaml_args['model_type'] = 'large'
    return yaml_args

yaml_args = setup_args_and_folder()

Constructing new results directory at results/BERT-disk-parse-distance-2020-5-12-13-34-35-861228/


/home/mli/anaconda3/envs/data-mining/lib/python3.6/site-packages/ipykernel_launcher.py:16: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  app.launch_new_instance()


# Requirements before running this code
- Corpus (conllx, for BERT)
    - Run `convert_splits_to_depparse.sh` (to get .conllx format)
    - Run `convert_conll_to_raw.py` (to get into raw text) 
- BERT-layer embeddings (.bert-layers.hdf5)
    - Run `convert_raw_to_bert.py` (Uses BERT to create bert-embeddings for ALL layers)
- Depth & Distance Params Path
    - Pretrained from data
- Ground Truths
    - Trees
        - Run Stanford CoreNLP's `ReadSentimentDataset` to get the ground truth trees
    - Sentiment
        - Run `apply_splits.py` to average all phrases' sentiments in the sentences

# Notes for generating the Tree data for SST
- This is for reading in the Ground Truth trees that is already given to us in SST
- We'll use Stanford's CoreNLP tools
- Run the ReadSentimentDataset `java -mx4g edu.stanford.nlp.sentiment.ReadSentimentDataset -inputDir data/SST-2/original -outputDir tmp/`
  - The ground truth already does subword partitions, so need to account for that

In [3]:
from nltk.tree import Tree

from reporter import WordPairReporter, WordReporter, prims_matrix_to_edges
from tqdm import tqdm
import copy

In [4]:
def read_trees(path):
    with open(path) as f:
        tree_lines = f.readlines()
        
    trees = [Tree.fromstring(treeline) for treeline in tree_lines]
    return trees

def read_sentiment_sentences(path):
    with open(path) as f:
        all_sentences = f.readlines()
        
        sentences, labels = [], []
        for pair in all_sentences:
            sentence, label = pair.split('\t')
            sentences.append(sentence)
            labels.append(label)
        return sentences, labels

In [5]:
data_base = Path('../../../data/SST-2')

train_path = data_base / 'sentence_splits' / 'train.tsv'
dev_path = data_base / 'sentence_splits' / 'dev.tsv'

# read in SST dataset
sst_trees_base = data_base / 'tree_format/'
gt_tree_train_path = sst_trees_base / 'train.txt'
gt_tree_dev_path = sst_trees_base / 'dev.txt'

train_sentiment, train_labels = read_sentiment_sentences(train_path)
dev_sentiment, dev_labels = read_sentiment_sentences(dev_path)

# Read into NTLK Trees
gt_train_trees = read_trees(gt_tree_train_path)
gt_dev_trees = read_trees(gt_tree_dev_path)

In [6]:
from finetune_bert_module import SST_Test

model = BertForSequenceClassification.from_pretrained('best_models/regression_training_pt2')
model = model.to(yaml_args['device'])
model.eval()

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

INFO:transformers.configuration_utils:loading configuration file best_models/regression_training_pt2/config.json
INFO:transformers.configuration_utils:Model config BertConfig {
  "_num_labels": 1,
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "directionality": "bidi",
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "min_length": 0,
  "model_type": "bert",
  "no_repeat_ngram_size": 0,
  "num_attention_heads": 16,
  "num_beams": 1,

In [7]:
# Calculate distance between the two trees
# one tree's format is from structural probes
# other tree's format is from the dataset (gt_*_trees)
# import importlib
# importlib.reload(eval_probes_on_dataset)

word_dists, word_depths, predicted_edges = eval_probes_on_dataset.use_probes(yaml_args, dev_sentiment, model, tokenizer)

[demoing]: 0it [00:00, ?it/s]

Constructing TwoWordPSDProbe
Constructing OneWordPSDProbe


[demoing]: 1101it [00:28, 38.62it/s]


In [13]:
# Calculate accuracy of the model
sigmoid = torch.nn.Sigmoid()
mse = 0
for idx, line in tqdm(enumerate(dev_sentiment), desc='Eval Dev Sentiment'):
    _, tokens_tensor, segments_tensors = eval_probes_on_dataset.prepare_sentence_for_bert(line, tokenizer)
    line_label = float(dev_labels[idx])

    device = yaml_args['device']
    tokens_tensor = tokens_tensor.unsqueeze(0).to(device)
    segments_tensors = segments_tensors.unsqueeze(0).to(device)
    with torch.no_grad():
        logits, encoded_layers = model(tokens_tensor, segments_tensors)
        probs = sigmoid(logits)
        print(probs)
        mse += (line_label - probs.item()) ** 2

print('Higher is bad', mse / len(dev_sentiment))        

Eval Dev Sentiment: 8it [00:00, 34.90it/s]

tensor([[0.0067]], device='cuda:0')
tensor([[0.0085]], device='cuda:0')
tensor([[0.0088]], device='cuda:0')
tensor([[0.0054]], device='cuda:0')
tensor([[0.0046]], device='cuda:0')
tensor([[0.0090]], device='cuda:0')
tensor([[0.0058]], device='cuda:0')
tensor([[0.0064]], device='cuda:0')


Eval Dev Sentiment: 16it [00:00, 37.06it/s]

tensor([[0.0072]], device='cuda:0')
tensor([[0.0105]], device='cuda:0')
tensor([[0.0055]], device='cuda:0')
tensor([[0.0382]], device='cuda:0')
tensor([[0.0205]], device='cuda:0')
tensor([[0.0085]], device='cuda:0')
tensor([[0.0054]], device='cuda:0')
tensor([[0.0042]], device='cuda:0')


Eval Dev Sentiment: 24it [00:00, 38.34it/s]

...

Eval Dev Sentiment: 320it [00:08, 39.76it/s]

tensor([[0.0040]], device='cuda:0')
tensor([[0.0045]], device='cuda:0')
tensor([[0.0042]], device='cuda:0')
tensor([[0.0049]], device='cuda:0')
tensor([[0.0070]], device='cuda:0')
tensor([[0.0235]], device='cuda:0')
tensor([[0.0587]], device='cuda:0')
tensor([[0.0113]], device='cuda:0')


Eval Dev Sentiment: 320it [00:08, 39.25it/s]


tensor([[0.0097]], device='cuda:0')
tensor([[0.0138]], device='cuda:0')


KeyboardInterrupt: 

In [9]:
def earliest_root(a, b):
    for i in range(min(len(a), len(b))):
        if a[i] != b[i]:
            return i
    return -1

def get_adj_matrix(cur_tree):
    num_words = len(cur_tree.leaves())
    all_paths = [cur_tree.leaf_treeposition(i) for i in range(num_words)]
    matrix = [[0] * num_words for _ in range(num_words)]

    for a_idx, a in enumerate(all_paths):
        for b_idx, b in enumerate(all_paths):
            if a_idx < b_idx:
                root = earliest_root(a, b)
                matrix[a_idx][b_idx] = len(a) + len(b) - 2 * root
                matrix[b_idx][a_idx] = len(a) + len(b) - 2 * root
    return matrix

In [10]:
def report_uuas(predicted_edges, trees, split_name):
    uspan_total = 0
    uspan_correct = 0
    total_sents = 0
    for predicted_edge, gt_tree in tqdm(zip(predicted_edges, trees), desc='[uuas,tikz]'):
        words = gt_tree.leaves()
        poses = gt_tree.leaves()
        gold_edges = get_adj_matrix(gt_tree)
        gold_edges = prims_matrix_to_edges(gold_edges, words, poses)
        pred_edges = predicted_edge

        gold_span = set([tuple(sorted(x)) for x in gold_edges])
        pred_span = set([tuple(sorted(x)) for x in pred_edges])
        uspan_correct += len(gold_span.intersection(pred_span))
        uspan_total += len(gold_edges)
        total_sents += 1

    uuas = uspan_correct / float(uspan_total)
    return uuas

uuas_score = report_uuas(predicted_edges, gt_dev_trees, 'dev')
print(uuas_score)

[uuas,tikz]: 1101it [00:01, 637.77it/s]

0.29806358541863176


In [11]:
def get_sentiments():
    pass

In [12]:
%%notify "result"
result = f"Finished SST: {uuas_score:.4f}"
print(result)

Finished SST: 0.2981
